In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-11.parquet

# Q1: Install Spark and PySpark

- Install Spark
- Run PySpark
- Create a local spark session
- Execute spark.version.

What's the output?

In [2]:
import pyspark

pyspark.__version__

'4.1.1'

# Q2: Yellow October 2024

Read the October 2024 Yellow into a Spark Dataframe.

Repartition the Dataframe to 4 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df = spark.read \
    .option("header", "true") \
    .parquet("yellow_tripdata_2025-11.parquet")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/23 20:41:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|cbd_congestion_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------------+
|       7| 2025-11-01 00:13:25|  2025-11-01 00:13:25|              1|         1.68|         1|                 N|          43|    

In [5]:
df=df.repartition(4)

df.write.parquet("yellow/2025/11/")

In [6]:
!du -h yellow/2025/11/*

  0B	yellow/2025/11/_SUCCESS
 25M	yellow/2025/11/part-00000-3aa279a6-0d5b-4059-857f-2fae1521efdb-c000.snappy.parquet
 25M	yellow/2025/11/part-00001-3aa279a6-0d5b-4059-857f-2fae1521efdb-c000.snappy.parquet
 25M	yellow/2025/11/part-00002-3aa279a6-0d5b-4059-857f-2fae1521efdb-c000.snappy.parquet
 25M	yellow/2025/11/part-00003-3aa279a6-0d5b-4059-857f-2fae1521efdb-c000.snappy.parquet


# Q3: Count records 

How many taxi trips were there on the 15th of November?

Consider only trips that started on the 15th of November.

In [7]:
from pyspark.sql import functions as F

df \
    .withColumn('pickup_date', F.to_date(df.tpep_pickup_datetime)) \
    .filter(F.col('pickup_date') == '2025-11-15') \
    .select(F.count('pickup_date')) \
    .show()

+------------------+
|count(pickup_date)|
+------------------+
|            162604|
+------------------+



# Q4: Longest trip

What is the length of the longest trip in the dataset in hours?

In [8]:
df \
    .withColumn('diff', 
    (F.unix_timestamp('tpep_dropoff_datetime') - F.unix_timestamp('tpep_pickup_datetime'))/3600) \
    .select("diff") \
    .orderBy(F.col("diff").desc()) \
    .show()

+------------------+
|              diff|
+------------------+
| 90.64666666666666|
| 76.94833333333334|
| 76.21388888888889|
| 69.28861111111111|
| 67.08055555555555|
| 63.36833333333333|
|56.382222222222225|
|48.147777777777776|
| 47.47833333333333|
| 45.44416666666667|
| 44.04333333333334|
|43.230555555555554|
|42.720555555555556|
|41.614444444444445|
| 41.50333333333333|
| 39.33305555555555|
|38.074444444444445|
| 37.91111111111111|
| 34.87027777777778|
| 30.53388888888889|
+------------------+
only showing top 20 rows


# Q5: User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

Answer: `4040`

# Q6: Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark:

```bash
wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
```

Using the zone lookup data and the Yellow October 2024 data, what is the name of the LEAST frequent pickup location Zone?

In [9]:
df_lookup = spark.read \
            .option("header", "true") \
            .csv("taxi_zone_lookup.csv")

df_lookup.createOrReplaceTempView("zone_lookup")
df.createOrReplaceTempView("trip_data")

df \
    .join(df_lookup, df.PULocationID == df_lookup.LocationID) \
    .groupBy("Zone") \
    .count() \
    .orderBy("count") \
    .show()

+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
|Eltingville/Annad...|    1|
|       Arden Heights|    1|
|       Port Richmond|    3|
|       Rikers Island|    4|
|   Rossville/Woodrow|    4|
|         Great Kills|    4|
| Green-Wood Cemetery|    4|
|         Jamaica Bay|    5|
|         Westerleigh|   12|
|New Dorp/Midland ...|   14|
|       West Brighton|   14|
|             Oakwood|   14|
|        Crotona Park|   14|
|       Willets Point|   15|
|Breezy Point/Fort...|   16|
|Saint George/New ...|   17|
|       Broad Channel|   18|
|     Mariners Harbor|   21|
|Heartland Village...|   22|
+--------------------+-----+
only showing top 20 rows
